In [7]:
%pip install nbformat
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns


/tmp/ipykernel_1747867/850107665.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import make_scorer, balanced_accuracy_score

def train_classifier(X_train, y_train, X_test):
  """
  Trains an MLP neural network model using a pipeline that includes:
  frequency encoding, one-hot encoding, and hyperparameter tuning.

  Parameters:
      X_train (pd.DataFrame): Training features.
      y_train (pd.Series or np.array): Training target values.
      X_test (pd.DataFrame): Test features.

  Returns:
      best_estimator: Best estimator from RandomizedSearchCV.
      test_predictions: Predicted labels for X_test.
  """
  
  # Construct pipeline:
  pipeline = Pipeline([
    ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('mlp', MLPClassifier(max_iter=300, random_state=42))
  ])

  # Define param grid for MLPClassifier
  param_distributions = {
    'mlp__hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__solver': ['adam'],
    'mlp__alpha': [0.0001, 0.001],
    'mlp__learning_rate': ['constant', 'adaptive']
  }

  # Balanced accuracy scorer
  balanced_acc = make_scorer(balanced_accuracy_score)

  randomized_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring=balanced_acc,
    verbose=3,
    n_jobs=-1
  )

  randomized_search.fit(X_train, y_train)

  print('Best parameters:', randomized_search.best_params_)
  print('Best cross-validation balanced accuracy:', randomized_search.best_score_)

  cv_scores = cross_val_score(
    randomized_search.best_estimator_,
    X_train, y_train,
    cv=5,
    scoring=balanced_acc,
    verbose=3
  )
  print('Generalization balanced accuracy (via cross_val_score):', cv_scores.mean())

  test_predictions = randomized_search.predict(X_test)

  # Save results to CSV
  df_preds = pd.DataFrame({
    'Id': range(1, len(test_predictions) + 1),
    'Outcome Type': test_predictions
  })
  df_preds.to_csv('neural_net_preds.csv', index=False)
  print('Predictions saved to neural_net_preds.csv')


  return randomized_search.best_estimator_, test_predictions

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder
# For Dog:
train_dog = df_train[df_train['animal_type'] == 'Dog'].copy()
X_train_dog = train_dog.drop(columns=['animal_type', 'outcome_type'])
y_train_dog = train_dog['outcome_type']

test_dog = df_test[df_test['animal_type'] == 'Dog'].copy()
X_test_dog = test_dog.drop(columns=['animal_type'])

# For Cat:
train_cat = df_train[df_train['animal_type'] == 'Cat'].copy()
X_train_cat = train_cat.drop(columns=['animal_type', 'outcome_type'])
y_train_cat = train_cat['outcome_type']

test_cat = df_test[df_test['animal_type'] == 'Cat'].copy()
X_test_cat = test_cat.drop(columns=['animal_type'])

## Encode targets with LabelEncoder
# Dog encoding
le_dog = LabelEncoder()
y_train_dog_encoded = le_dog.fit_transform(y_train_dog)

# Cat encoding
le_cat = LabelEncoder()
y_train_cat_encoded = le_cat.fit_transform(y_train_cat)

print("Training model for Dog data:")
best_estimator_dog, dog_predictions_encoded = train_classifier(X_train_dog, y_train_dog_encoded, X_test_dog)
dog_predictions = le_dog.inverse_transform(dog_predictions_encoded)

print("\nTraining model for Cat data:")
best_estimator_cat, cat_predictions_encoded = train_classifier(X_train_cat, y_train_cat_encoded, X_test_cat)
cat_predictions = le_cat.inverse_transform(cat_predictions_encoded)

combine_predictions(dog_predictions, cat_predictions, X_test_dog, X_test_cat)

Training model for Dog data:
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 2/5] END mlp__activation=relu, mlp__alpha=0.001, mlp__hidden_layer_sizes=(100, 50), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.372 total time= 8.5min
[CV 5/5] END mlp__activation=relu, mlp__alpha=0.001, mlp__hidden_layer_sizes=(100, 50), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.371 total time= 9.6min
[CV 3/5] END mlp__activation=relu, mlp__alpha=0.001, mlp__hidden_layer_sizes=(100, 50), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.340 total time=10.3min
[CV 4/5] END mlp__activation=relu, mlp__alpha=0.001, mlp__hidden_layer_sizes=(100, 50), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.363 total time=12.6min
[CV 1/5] END mlp__activation=relu, mlp__alpha=0.001, mlp__hidden_layer_sizes=(100, 50), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.353 total time=13.0min
[CV 2/5] END mlp__activation=relu, mlp__alpha=0.0001, mlp__hidden_layer_siz